<a href="https://colab.research.google.com/github/easypanda/Handson-ML2/blob/master/Support_Vector_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Support Vector Machines

A Support Vector Machine (SVM) is a powerful and versatile Machine Learning model, capable of performing linear or nonlinear classification, regression , and even outlier detection. SVMs are particulary well suited for classification of complex small -or medium- sized datasets.

## Linear SVM Classification

You can think of an SVM classifier as fitting the widest possible street (represented by the parallel dashed lines) between the classes. This is called **large margin classification**. Notice that adding more training instances "off the street" will not affect the decision boundary at all: it is fully determined by the instances located on the edge of the street. These instances are called the **Support Vectors**.

/!\ : SVMs are sensible to the feature scales so we should do feature scaling each time.

## Soft Margin Classification

If we strictly impose that all instances must be off the street and on the right side, this is called **hard margin classification**.
This has two main issues:
* It only works if the data are linearly seperable.
* It is sensitive to outliers.

We choose use a more flexible model that ensure the good balance between keeping the street as large as possible and limiting the *margin violations*.
This is called **Soft Margin Classification**.

When creating a model with Scikit-Learn, we can specify the number of hyperparameters including C. if C is set to a low value, we enable a large margin and therefore more margin violations whereas with a high C, a fewer margin violations. Sometimes it's more sutable to have more margin violations to ensure a better generalization of the model.

/!\ If the model is overfitting, we can try regularizing by reducing C.

In [0]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [0]:
iris = datasets.load_iris()
X = iris["data"][:,(2,3)] #Petal length and petal width columns
y = (iris["target"] == 2).astype(np.float64) #Just Iris virginica

In [0]:
svm_clf = Pipeline([
                    ("scaler",StandardScaler()),
                    ("linear_svc",LinearSVC(C=1,loss="hinge")),
])

In [6]:
svm_clf.fit(X,y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [7]:
svm_clf.predict([[5.5,1.7]])

array([1.])

**/!\** Unline Logistic Regression, SVM does not output probabilities for each class.

We could also use the SVC class with a liner kernel (SVC(kernel="linear",C=1)) or with the SGDClassifier (SGDClassifier(loss="hinge",alpha=1/(m*c))). It does not converge as fast as the LinearSVC class but it can be useful to handle online classification tasks or huge datasets that do not fit in memory (out-of-core training).

Loss hyperparameter should be on "hinge" and for better performance, dual hyperparameter should be set on False.

## Nonlinear SVM Classification

One approche to handle nonlinear datasets is to add more features, such as polynomial features.

In [0]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [0]:
X,y = make_moons(n_samples=100,noise=0.15)
polynomial_svm_clf = Pipeline([
                               ('poly_features',PolynomialFeatures(degree=3)),
                               ('scaler',StandardScaler()),
                               ('svm_clf',LinearSVC(C=10,loss="hinge"))
])

In [10]:
polynomial_svm_clf.fit(X,y)

Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

## Polynomial Kernel

Adding a polynomial features is simple to implement and can work great with all sorts of Machine Learning algorithms (not just SVM). That said, at a low polynomial degree, this method cannot deal with very complex datasets, and with a high polynomial degree it creates a huge number of features, making the model too slow.

But with SVM, you can apply a technique called **kernel trick** to make possible to have the same result as if you had added many polynomial features, even with very high-degree polynomials without actually adding them.

In [0]:
from sklearn.svm import SVC

In [0]:
poly_kernel_svm_clf = Pipeline([
                                ("scaler",StandardScaler()),
                                ("svm_clf",SVC(kernel="poly",degree=3,coef0=1,C=5))
])

In [13]:
poly_kernel_svm_clf.fit(X,y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=5, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

This SVM classifier uses a third-degree polynomial kernel.
If the model is overfitting, we can decrease the polynomial degree and if it's underfitting we can increase it.
The hyperparemeter coef0 controls how much the model is unfluenced by high-degree polynomials versus low-degree polynomials.

## Similarity Features

Another technique to tackle nonlinear problems is to add features computed using a **similarity function** which measures how much each instance resembles a particular **landmark**.

One similarity function is the Gaussian Radial Basis Function (RBF).

This is a bell-shaped function varying from 0 (very far from the landmark) to 1 (the landmark).

To select the landmarks, the simplest approach is to createa landmark at the location of each and every instance in the dataset. Doing that creates many dimensions and thus increases the chances that the transformed training set will be linearly seperable. The downside is that a training set with m instances and n features will be transformed into a training set with m instances and m features (assuming that the original features have been dropped). 

## Guassian RBF Kernel

The kernel trick does its SVM magic, making it possible to obtain a similar result as if you added many similarity features.

In [0]:
rbf_kernel_svm_clf = Pipeline([
                               ('scaler',StandardScaler()),
                               ("svm_clf",SVC(kernel="rbf",gamma=5,C=0.001))
])

In [15]:
rbf_kernel_svm_clf.fit(X,y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

The hyperparameter gamma:
* Makes the bell-shaped curve narrower if increased.
* The result is each instance's range of influence is smaller.

Conversely, a small gamma value makes the bell-shaped curve wider: Instances have a larger range of influence, and the decision boundary ends up smoother.

It acts like a regularization hyperparameter: if the model is overfitting, we should reduce it; if it's underfitting, we should increase it (likewise the hyperparameter C).

Other kernels exist but they are used more rarely (like the ones specialized for specific data structures, like string kernels for documents or DNA sequences).

As a rule of thumb, it should be always tried first the LinearSVC (as it's faster than the SVC(kernel="linear"), especially if the training set is very large or plenty of features. If it's not too large, the Gaussian RBF kernel can be also tried.

# Computational Complexity

The LinearSVC does not support the kernel trick, but it scales almost linearly with the number of training instances and the number of features. Its training time complexity is roughly O (m x n). The algorithm takes longer if you require very high precision. This is controlled by the tolerance hyperparameter tol in Scikit-Learn. In most classification, the default tolerance is fine.

The SVC class can be dreadfully slow when the number of training instances gets large ( because training complexity time is between O (m^2 * n ) and (m^3 * n )).
This algorithm is perfect for complex small or medium-sized training sets. It scales well with the number of features, especially with sparse features (when each instance has few nonzero features).

# SVM Regression

To use the SVMs for regression instead of classifications, the trick is to reverse the objective. Instead of trying to fit the largest possible street between two classes while limiting margin violations, SVM Regression tries to fit as many instances as possible on the street while limiting margin violations (instance off the street). The width of the street is controlled by a hyperparameter, tol.

Adding more training instance within the margin does not affect the model's prediction; thus the model is said to be tol-insensitive.

In [0]:
from sklearn.svm import LinearSVR

In [17]:
svm_reg = LinearSVR(epsilon = 1.5)
svm_reg.fit(X,y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

To tackle nonlinear regression tasks, you can use a kernelized SVM model.

In [0]:
#Reminder : Small C Value = High regularization, high C value = Low regularization.

In [0]:
from sklearn.svm import SVR

In [20]:
svm_poly_reg = SVR(kernel="poly",degree=2,C=100,epsilon=0.1)
svm_poly_reg.fit(X,y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

 The SVR Class is the regression equivalent of the SVC class, and the LinearSVR is the equivalent of the LinearSVC Class. The LinearSVR class scales linearly with the size of the training set, while the SVR class gets too slow if the training set grows large ( just like the SVC class).

*The SVMs can also be used for outliers detection.*

# Under the hood : Decision function and Predictions

The Linear SVM Classifier model predicts the class of a new instance **x** by simply computing the decision function **W(feature weights)^T X + b (bias term)**. If the result if positive, the predicted class is the positive class (1), if negative, the predicted class is the negative class (0).

**Training a linear SVM classifier means finding the values of w and b that make this margin as wide as possible while avoiding margin violations (hard margin) or limiting margin (soft margin).**

# Training Objective

Dividing the slope by 2 will multiply the margin by 2. The smaller the weight vector w, the larger the margin.

We want to minimize ||w|| to get a larger margin but also avoid any margin violations (hard margin) then we need the decision function to be greater than 1 for all positive training instances and lower than -1 for negative training instances.


For the soft margin objective, we need to introduce a slack variable for each instance. We have now two conflicting objectives: make the slack variables as small as possble to reduce the margin violations, and make 1/2*W^T*W as small as possible to increase the margin. This is where the hyperparameter C comes in: it allows us to define the trade-off between these two objectives.

# Quadratic Programming

The hard margin and soft margin problems are both convex quadratic optimization problems with linear constraints. Such problems are known as Quadratic Programming (QP) problems.

One way to train a hard margin linear SVM classifier is to use an off-the-shelf QP solver and pass it the preceding parameters.

To use the kernel trick, we are going to look at a different constrained optimization problem.

# The Dual Problem

Given a constrained optimization problem, known as the primal problem, it is possible to express a different but closely related problem, called its dual problem. 

The dual problem is fasterto solve than the primal one when the number of training instances is smaller than the number of features and makes the kernel trick possible.

# Kernelized SVMs

In Machine-Learning, a *kernel* is a function capable of computing the dot product (/)(a)^T * (/)(b) based only on the original vectors a and b, without having to compute the transformations (/). Some of the most common kernels are linear, polynomial, gaussian RBF, sigmoid.

# Online SVMs

As a reminder, online learning means learning incrementally, typically as new instances arrive.

For linear SVM Classifiers,one method for implementing an online SVM classifier is to use Gradient Descent (using SGDClassifier) to minimize the cost function which is derived from the primal proble. Unfortunately, Gradient Descent converges much more slowly than the methods based on QP.

For large-scale nonlinear problems, you may want to consider using neural networks instead.

**Hinge Loss** : The function max(0,1 - t) is called the *hinge loss* function and it is equal to zero when t>=1. Its derivative is equal to -1 if t <1 and 0 if t >1. It is not differentiable at t=1, but just like Lasso Regression, we can use Gradient Descent using any subderivative at t=1 (any value between -1 and 0).